In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout

In [4]:
df = pd.read_csv('Churn_Modelling.csv', index_col='RowNumber')

In [5]:
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.shape

(10000, 13)

In [7]:
X_columns = df.columns.tolist()[2:12]
Y_columns = df.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1.1+ MB


In [8]:
X = df[X_columns].values 
Y = df[Y_columns].values

In [11]:
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

In [15]:
pipeline = Pipeline([
    ('Categorizer', ColumnTransformer(
        [
            ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
            ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
        ], 
        remainder = 'passthrough', n_jobs = 1
    )),
    ('Normalizer', StandardScaler())
])

In [16]:
X = pipeline.fit_transform(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [20]:
#Initialize ANN
classifier = Sequential()

In [21]:
#Add input layer and hidden layer
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [22]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [23]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [24]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [25]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [26]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 2s - loss: 0.5463 - accuracy: 0.7889 - val_loss: 0.4713 - val_accuracy: 0.7950 - 2s/epoch - 7ms/step
Epoch 2/200
225/225 - 0s - loss: 0.4844 - accuracy: 0.7954 - val_loss: 0.4425 - val_accuracy: 0.7950 - 332ms/epoch - 1ms/step
Epoch 3/200
225/225 - 0s - loss: 0.4659 - accuracy: 0.7949 - val_loss: 0.4287 - val_accuracy: 0.7962 - 331ms/epoch - 1ms/step
Epoch 4/200
225/225 - 0s - loss: 0.4573 - accuracy: 0.7968 - val_loss: 0.4215 - val_accuracy: 0.7975 - 319ms/epoch - 1ms/step
Epoch 5/200
225/225 - 0s - loss: 0.4492 - accuracy: 0.8010 - val_loss: 0.4167 - val_accuracy: 0.8037 - 330ms/epoch - 1ms/step
Epoch 6/200
225/225 - 0s - loss: 0.4473 - accuracy: 0.8029 - val_loss: 0.4134 - val_accuracy: 0.8050 - 403ms/epoch - 2ms/step
Epoch 7/200
225/225 - 0s - loss: 0.4414 - accuracy: 0.8064 - val_loss: 0.4087 - val_accuracy: 0.8075 - 328ms/epoch - 1ms/step
Epoch 8/200
225/225 - 0s - loss: 0.4365 - accuracy: 0.8075 - val_loss: 0.4049 - val_accuracy: 0.8150 - 301ms/epoch - 1ms/

In [28]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 0s 1ms/step
[[0.36139   ]
 [0.3416382 ]
 [0.21647947]
 [0.05846361]
 [0.13060148]]


In [29]:
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [30]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1518   77]
 [ 218  187]]


In [31]:
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

85.25 % of data was classified correctly
